In [1]:
# Loading in packages
from PIL import Image
import numpy as np
import tensorflow as tf
import cv2
from tensorflow import keras

In [26]:
# Loading in video and variables
class Edit:
    def __init__(self,video_data_path):
        self.video_data_path = video_data_path
        self.vidcap = cv2.VideoCapture(self.video_data_path)
        self.frame_count = int(self.vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
        self.current_frame = 0
        self.start_frame = None
        self.list_start_frame =[]
        self.list_end_frame =[]
        self.end_frame = None
        self.end_video =False
        self.fps = 60
        # Loading in model
        self.model = keras.models.load_model('model_22_01_11_notts_tuam')
    
    def read_frame(self,frame):
        self.vidcap.set(1,frame)
        ret, image = self.vidcap.read()
        imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        temp_image = Image.fromarray(imageRGB)
        # temp_image = Image.open("temp.jpg")
        temp_image = temp_image.resize((256, 256))
        temp_array = np.asarray(temp_image)
        output = np.empty((1,256,256,3))
        output[0] = temp_array
        return output

    def kayak_check(self,frame,start):
        test_frame = self.read_frame(frame)
        temp_predict = self.model.predict(test_frame)
        current_predict = temp_predict[0,1]
        if start:
            if current_predict > 10:
                return True, frame
            elif current_predict <10 and current_predict > 5:
                return False, (frame+1)
            else: 
                return False, (frame+60)
        else:
            if current_predict > 10:
                return False, (frame+60)
            elif current_predict <10 and current_predict > 5:
                return False, (frame+1)
            else: 
                return True, frame


In [28]:
path_to_vid = './Video_source/Tuam_video_3.MP4'
my_edit = Edit(path_to_vid)
while not my_edit.end_video:
    while my_edit.start_frame == None and  my_edit.end_video == False :
        if my_edit.current_frame >= my_edit.frame_count:
            my_edit.end_video = True
        else:
            start, my_edit.current_frame = my_edit.kayak_check(my_edit.current_frame,True)
            if start:
                my_edit.start_frame = my_edit.current_frame
                my_edit.current_frame += 30
    print(my_edit.start_frame)
    count = 0
    while my_edit.end_frame == None and  my_edit.end_video == False :
        if my_edit.current_frame >= my_edit.frame_count:
            my_edit.end_frame =my_edit.frame_count-1
            my_edit.end_video = True
            break
        else:
            end, my_edit.current_frame = my_edit.kayak_check(my_edit.current_frame,False)
            if end:
                count += 1
                if count > 3:
                    my_edit.end_frame = my_edit.current_frame
                    end = False 
                my_edit.current_frame += 20

    print(my_edit.end_frame)
    
    if my_edit.start_frame != None and my_edit.end_frame != None:
        my_edit.list_start_frame.append(my_edit.start_frame)
        my_edit.list_end_frame.append(my_edit.end_frame)
        start_time = my_edit.start_frame/my_edit.fps
        end_time = my_edit.end_frame/my_edit.fps
        print('Start time: ',start_time)
        print('End time: ',end_time)
        my_edit.start_frame = None 
        my_edit.end_frame = None
    else:
        my_edit.end_video = True



0
270
Start time:  0.0
End time:  4.5
1430
3624
Start time:  23.833333333333332
End time:  60.4
4004
4334
Start time:  66.73333333333333
End time:  72.23333333333333
4427
4962
Start time:  73.78333333333333
End time:  82.7
5182
5540
Start time:  86.36666666666666
End time:  92.33333333333333
5980
7879
Start time:  99.66666666666667
End time:  131.31666666666666
7907
8117
Start time:  131.78333333333333
End time:  135.28333333333333
9217
10467
Start time:  153.61666666666667
End time:  174.45
10869
12314
Start time:  181.15
End time:  205.23333333333332
12514
14049
Start time:  208.56666666666666
End time:  234.15
14969
16388
Start time:  249.48333333333332
End time:  273.1333333333333
16948
18968
Start time:  282.46666666666664
End time:  316.1333333333333
19528
21265
Start time:  325.46666666666664
End time:  354.4166666666667
None
None


In [47]:
# my_edit.frame_count
import math
my_edit.list_start_frame
list_start_time = []
list_end_time = []

for time in my_edit.list_start_frame:
    list_start_time.append(time/my_edit.fps)
for time in my_edit.list_end_frame:
    list_end_time.append(time/my_edit.fps)

for item in range(len(list_start_time)):
    print('Start_time = '+str(math.floor(list_start_time[item]/60)) + ':' + str(round(((list_start_time[item]) % 60),2)) +'    End_time = ' +str(math.floor(list_end_time[item]/60)) + ':' + str(round(((list_end_time[item]) % 60),2)))

Start_time = 0:0.0    End_time = 0:4.5
Start_time = 0:23.83    End_time = 1:0.4
Start_time = 1:6.73    End_time = 1:12.23
Start_time = 1:13.78    End_time = 1:22.7
Start_time = 1:26.37    End_time = 1:32.33
Start_time = 1:39.67    End_time = 2:11.32
Start_time = 2:11.78    End_time = 2:15.28
Start_time = 2:33.62    End_time = 2:54.45
Start_time = 3:1.15    End_time = 3:25.23
Start_time = 3:28.57    End_time = 3:54.15
Start_time = 4:9.48    End_time = 4:33.13
Start_time = 4:42.47    End_time = 5:16.13
Start_time = 5:25.47    End_time = 5:54.42


In [6]:
# from PIL import Image
# from matplotlib import cm
# path = './Video_source/func_test_02_mix.mp4'
# vid = cv2.VideoCapture(path)
# t , pic = vid.read()
# # im = Image.fromarray(np.uint8(cm.gist_earth(pic)*255))
# # pic.shape
# imageRGB = cv2.cvtColor(pic, cv2.COLOR_BGR2RGB)
# im = Image.fromarray(imageRGB)
# # im = Image.fromarray(pic,mode="RGBa")
# im

In [42]:
round(((list_start_time[1]) % 60),2)

23.83